In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.resnet152V2 import ResNet152V2
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Dataset/Train'
valid_path = '/content/drive/MyDrive/Dataset/Test'

In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
import tensorflow
resnet152V2 =tensorflow.keras.applications.ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

234545216/234545216 [==============================] - 1s 0us/step


In [5]:
# don't train existing weights
for layer in resnet152V2.layers:
    layer.trainable = False

In [6]:
  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Dataset/Train/*')

In [7]:
# our layers - you can add more if you want
x = Flatten()(resnet152V2.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet152V2.input, outputs=prediction)

In [9]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Dataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 705 images belonging to 6 classes.


In [13]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Dataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 78 images belonging to 6 classes.


In [1]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: ignored

In [ ]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_resnet152V2.h5')

In [ ]:
y_pred = model.predict(test_set)

3/3 [==============================] - 19s 5s/step


In [ ]:
y_pred

array([[5.83596176e-08, 9.99995470e-01, 3.34636830e-26, 2.40440094e-35,
        3.78455707e-06, 7.17490934e-07],
       [0.00000000e+00, 3.69756146e-24, 6.19166038e-24, 0.00000000e+00,
        1.23283009e-15, 1.00000000e+00],
       [1.09870328e-26, 9.99547064e-01, 4.52939566e-04, 0.00000000e+00,
        0.00000000e+00, 1.03281526e-08],
       [9.79815006e-01, 2.01849360e-02, 1.55656635e-18, 0.00000000e+00,
        8.92140611e-34, 1.03436669e-36],
       [3.97833227e-31, 1.04086893e-27, 0.00000000e+00, 1.00000000e+00,
        3.17556092e-28, 0.00000000e+00],
       [1.00000000e+00, 3.30338740e-26, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        3.37414760e-26, 5.43130179e-26],
       [7.99989415e-25, 1.00000000e+00, 5.26523894e-35, 0.00000000e+00,
        0.00000000e+00, 

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

AxisError: ignored

In [ ]:
y_pred

array([1, 5, 1, 0, 3, 0, 2, 3, 1, 4, 0, 2, 2, 2, 1, 3, 5, 0, 2, 2, 1, 2,
       2, 1, 1, 4, 5, 4, 4, 5, 1, 1, 5, 3, 1, 2, 2, 5, 0, 4, 2, 0, 2, 4,
       1, 2, 3, 4, 3, 1, 2, 2, 1, 3, 3, 0, 1, 3, 4, 4, 2, 2, 1, 2, 4, 4,
       1, 1, 1, 1, 1, 2, 3, 3, 4, 2, 2, 3])

In [ ]:

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_resnet152V2.h5')

In [ ]:
model

In [ ]:
img=image.load_img('/content/drive/MyDrive/Dataset/Test/Leaf_Spot/Copy of Leaf_Spot_original_IMG_2493.JPG_c85ee449-b50a-4e56-9157-0815a837eab4.JPG',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

array([[[131., 121.,  94.],
        [155., 130., 108.],
        [207., 179., 158.],
        ...,
        [ 90.,  77.,  60.],
        [120., 101.,  87.],
        [141., 135., 111.]],

       [[249., 236., 204.],
        [199., 173., 140.],
        [170., 140., 106.],
        ...,
        [ 40.,  23.,   5.],
        [ 55.,  41.,  28.],
        [ 67.,  53.,  40.]],

       [[253., 254., 238.],
        [189., 176., 142.],
        [193., 160., 129.],
        ...,
        [137., 109.,  85.],
        [ 57.,  43.,  34.],
        [123., 106.,  86.]],

       ...,

       [[237., 213., 177.],
        [254., 251., 208.],
        [254., 250., 203.],
        ...,
        [ 45.,  36.,  31.],
        [ 33.,  24.,  19.],
        [  8.,   3.,   0.]],

       [[230., 206., 172.],
        [248., 229., 196.],
        [245., 232., 188.],
        ...,
        [ 47.,  42.,  38.],
        [101.,  97.,  86.],
        [ 20.,  11.,   6.]],

       [[217., 191., 164.],
        [233., 215., 175.],
        [203., 1

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
x=x/255

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

1/1 [==============================] - 2s 2s/step


array([[3.3039346e-12, 4.7840027e-11, 4.1218332e-06, 1.2461794e-06,
        9.9999464e-01, 1.0648280e-11]], dtype=float32)

In [ ]:
img_data


array([[[[-0.9959708 , -0.99627835, -0.9971088 ],
         [-0.9952326 , -0.99600154, -0.9966782 ],
         [-0.9936332 , -0.99449444, -0.9951403 ],
         ...,
         [-0.99723184, -0.99763167, -0.9981546 ],
         [-0.9963091 , -0.9968935 , -0.9973241 ],
         [-0.9956632 , -0.99584776, -0.9965859 ]],

        [[-0.9923414 , -0.9927412 , -0.9937255 ],
         [-0.99387926, -0.994679  , -0.995694  ],
         [-0.99477124, -0.995694  , -0.99673975],
         ...,
         [-0.9987697 , -0.99929255, -0.9998462 ],
         [-0.99830836, -0.99873894, -0.9991388 ],
         [-0.9979392 , -0.9983699 , -0.9987697 ]],

        [[-0.9922184 , -0.9921876 , -0.9926797 ],
         [-0.9941869 , -0.9945867 , -0.99563247],
         [-0.9940638 , -0.9950788 , -0.9960323 ],
         ...,
         [-0.99578625, -0.9966474 , -0.9973856 ],
         [-0.99824685, -0.99867743, -0.99895424],
         [-0.99621683, -0.99673975, -0.99735487]],

        ...,

        [[-0.9927105 , -0.9934487 , -0

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)
a==1

1/1 [==============================] - 0s 36ms/step


array([False])

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.12.0'